In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_auc_score, r2_score

from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [3]:
data =  pd.read_excel('PahaW.xlsx')
data.shape

(75, 10)

In [4]:
data.head()

,ID,Nationality,Sex,Disease,PD status,Age,Dominant hand,LED,UPDRS V,Length of PD
0,1,Czech,F,PD,ON,68,R,1115.0,2.0,6.0
1,2,Czech,F,PD,ON,78,R,2110.0,2.0,8.0
2,3,Czech,F,PD,ON,69,R,1556.6,2.0,7.0
3,4,Czech,F,PD,ON,79,R,1691.0,2.0,12.0
4,5,Czech,F,PD,ON,69,R,600.0,2.0,2.0


In [5]:

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['Nationality','Sex','Disease','PD status','Dominant hand','ID'], axis=1),
    data['PD status'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((52, 4), (23, 4))

In [6]:
# remove correlated features to reduce the feature space

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

corr_features = correlation(X_train, 0.8)
print('correlated features: ', len(set(corr_features)) )

correlated features:  0


In [7]:
# remove correlated features
X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)

X_train.shape, X_test.shape

((52, 4), (23, 4))

In [10]:
sfs = SFS(RandomForestRegressor(n_estimators=10, n_jobs=4, random_state=10), 
           k_features=20, 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='r2',
           cv=2)

sfs = sfs.fit(np.array(X_train), y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


TypeError: '<' not supported between instances of 'float' and 'str'

In [12]:
def run_randomForests(X_train, X_test, y_train, y_test):
    
    rf = RandomForestClassifier(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)

    print('Train set')
    pred = rf.predict_proba(X_train)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
    
    print('Test set')
    pred = rf.predict_proba(X_test)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

In [14]:
# evaluate performance of algorithm built
# using selected features
selected_feat = X_train.columns[list(sfs.k_feature_idx_)]
selected_feat

run_randomForests(X_train[selected_feat],
                  X_test[selected_feat],
                  y_train, y_test)

TypeError: 'NoneType' object is not iterable